In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from keras.datasets import imdb 
from keras.layers import *
from keras.models import *
from keras import backend as K
from sklearn.model_selection import train_test_split

In [2]:
n_unique_words = 10000
(x_train, y_train),(x_test, y_test) = imdb.load_data(num_words=n_unique_words)

In [3]:
maxlen = 200
batch_size=128
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
y_train = np.array(y_train)
y_test = np.array(y_test) 
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2022)

In [4]:
class Attention(Layer):
  def __init__(self, return_sequences=True):
      self.return_sequences = return_sequences
      super(Attention,self).__init__()

  def build(self, input_shape):
      self.W=self.add_weight(name="att_weight", shape=(input_shape[-1],1),
                              initializer="normal")
      self.b=self.add_weight(name="att_bias", shape=(input_shape[1],1),
                              initializer="zeros")

      super(Attention,self).build(input_shape)
  def call(self, x):
      e = K.tanh(K.dot(x,self.W)+self.b)
      a = K.softmax(e, axis=1)
      output = x*a
      if self.return_sequences:

          return output
      return K.sum(output, axis=1)

In [5]:
model = Sequential()
model.add(Embedding(n_unique_words, 128, input_length=maxlen))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Attention(return_sequences=True))
model.add(LSTM(32))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 128)          1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 128)          98816     
_________________________________________________________________
attention (Attention)        (None, 200, 128)          328       
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 1,399,785
Trainable params: 1,399,785
Non-trainable params: 0
______________________________________________

In [6]:
history3d=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=12,
          validation_data=(x_val, y_val))

Epoch 1/12
157/157 [==============================] - 31s 108ms/step - loss: 0.6770 - accuracy: 0.5627 - val_loss: 0.5941 - val_accuracy: 0.7296
Epoch 2/12
157/157 [==============================] - 14s 87ms/step - loss: 0.4541 - accuracy: 0.7989 - val_loss: 0.3742 - val_accuracy: 0.8324
Epoch 3/12
157/157 [==============================] - 13s 80ms/step - loss: 0.2951 - accuracy: 0.8875 - val_loss: 0.3489 - val_accuracy: 0.8468
Epoch 4/12
157/157 [==============================] - 13s 81ms/step - loss: 0.2020 - accuracy: 0.9298 - val_loss: 0.3471 - val_accuracy: 0.8690
Epoch 5/12
157/157 [==============================] - 14s 91ms/step - loss: 0.1448 - accuracy: 0.9543 - val_loss: 0.3821 - val_accuracy: 0.8726
Epoch 6/12
157/157 [==============================] - 14s 92ms/step - loss: 0.1059 - accuracy: 0.9679 - val_loss: 0.4565 - val_accuracy: 0.8610
Epoch 7/12
157/157 [==============================] - 14s 92ms/step - loss: 0.0791 - accuracy: 0.9776 - val_loss: 0.5078 - val_accuracy

In [8]:
cross_entropy = model.evaluate(x_test, y_test)
print("Test loss and accuracy: ",cross_entropy)

782/782 [==============================] - 18s 23ms/step - loss: 0.6891 - accuracy: 0.8475
Test loss and accuracy:  [0.6891041994094849, 0.8475199937820435]


In [9]:
y_preds=model.predict(x_test)
print("First 10 test results:")
for i in range(10):
    print("Prediction: {} Actual: {}".format(y_preds[i],y_test[i]))

First 10 test results:
Prediction: [0.04758357] Actual: 0
Prediction: [0.9995814] Actual: 1
Prediction: [0.12353067] Actual: 1
Prediction: [0.36380428] Actual: 0
Prediction: [0.9995702] Actual: 1
Prediction: [0.99945325] Actual: 1
Prediction: [0.99951017] Actual: 1
Prediction: [0.00077298] Actual: 0
Prediction: [0.99961734] Actual: 0
Prediction: [0.9996784] Actual: 1
